In [1]:
import pandas as pd
import numpy as np

In [3]:
train = pd.read_csv("train.csv")

In [10]:
train_label = pd.read_csv("train_labels.csv")

In [4]:
train.head(1)

,event_id,game_session,timestamp,event_data,installation_id,event_count,event_code,game_time,title,type,world
0,27253bdc,45bb1e1b6b50c07b,2019-09-06T17:53:46.937Z,"{""event_code"": 2000, ""event_count"": 1}",0001e90f,1,2000,0,Welcome to Lost Lagoon!,Clip,NONE


In [6]:
# identifier for each session
train.game_session

0           45bb1e1b6b50c07b
1           17eeb7f223665f53
2           0848ef14a8dc6892
3           0848ef14a8dc6892
4           0848ef14a8dc6892
                  ...       
11341037    c09b94eebfdf50a6
11341038    6ad8ab25003ef1b0
11341039    71904dca23b982c6
11341040    2267b18f069e875a
11341041    f31950f549d7c242
Name: game_session, Length: 11341042, dtype: object

In [26]:
train.event_data

0                      {"event_code": 2000, "event_count": 1}
1                      {"event_code": 2000, "event_count": 1}
2           {"version":"1.0","event_count":1,"game_time":0...
3           {"description":"Let's build a sandcastle! Firs...
4           {"description":"Let's build a sandcastle! Firs...
                                  ...                        
11341037    {"description":"Tap another dinosaur.","identi...
11341038               {"event_code": 2000, "event_count": 1}
11341039               {"event_code": 2000, "event_count": 1}
11341040               {"event_code": 2000, "event_count": 1}
11341041               {"event_code": 2000, "event_count": 1}
Name: event_data, Length: 11341042, dtype: object

In [7]:
# user id
train.installation_id

0           0001e90f
1           0001e90f
2           0001e90f
3           0001e90f
4           0001e90f
              ...   
11341037    fffc0583
11341038    fffc0583
11341039    fffc0583
11341040    fffc0583
11341041    fffc0583
Name: installation_id, Length: 11341042, dtype: object

In [23]:
# what is event?
train.event_count

0            1
1            1
2            1
3            2
4            3
            ..
11341037    66
11341038     1
11341039     1
11341040     1
11341041     1
Name: event_count, Length: 11341042, dtype: int64

In [27]:
# an event can look like this
train.event_data[3]

'{"description":"Let\'s build a sandcastle! First, fill up your mold with sand! You can use the shovel here. The mold gives the sand its shape!","identifier":"Dot_LetsSandcastle,Dot_FillMold,Dot_MoldShape","media_type":"audio","total_duration":6758,"event_count":2,"game_time":53,"event_code":3010}'

In [36]:
print(f"""
event codes: 

{train.event_code.unique()}

and there are {train.event_code.nunique()} many events types
""")


event codes: 

[2000 3010 3110 4070 4090 4030 4035 4021 4020 4010 2080 2083 2040 2020
 2030 3021 3121 2050 3020 3120 2060 2070 4031 4025 5000 5010 2081 2025
 4022 2035 4040 4100 2010 4110 4045 4095 4220 2075 4230 4235 4080 4050]

and there are 42 many events types



# Peek data: game_time

## Takeaway (game_time) 
record does not include clip watching time

In [38]:
# session time? or "game" time?
train.game_time

0                0
1                0
2                0
3               53
4             6972
             ...  
11341037    118615
11341038         0
11341039         0
11341040         0
11341041         0
Name: game_time, Length: 11341042, dtype: int64

In [47]:
train[train.type == "Clip"]["game_time"].unique()

array([0])

In [53]:
train[train.type == 'Activity']["game_time"].nunique()

584525

In [54]:
train[train.type == 'Game']["game_time"].nunique()

663287

In [55]:
train[train.type == 'Assessment']["game_time"].nunique()

137710

In [ ]:
train[train.type == 'Assessment']["game_time"].nunique()

In [48]:
train.type.unique()

array(['Clip', 'Activity', 'Game', 'Assessment'], dtype=object)

In [57]:
train.title.unique()

array(['Welcome to Lost Lagoon!', 'Magma Peak - Level 1',
       'Sandcastle Builder (Activity)', 'Scrub-A-Dub',
       'Magma Peak - Level 2', 'Dino Drink', 'Tree Top City - Level 1',
       'Ordering Spheres', 'Watering Hole (Activity)', 'Slop Problem',
       'All Star Sorting', 'Costume Box', 'Fireworks (Activity)',
       '12 Monkeys', 'Tree Top City - Level 2',
       'Flower Waterer (Activity)', "Pirate's Tale",
       'Mushroom Sorter (Assessment)', 'Air Show', 'Treasure Map',
       'Tree Top City - Level 3', 'Crystals Rule', 'Rulers',
       'Bug Measurer (Activity)', 'Bird Measurer (Assessment)',
       'Bubble Bath', 'Bottle Filler (Activity)', 'Dino Dive',
       'Crystal Caves - Level 1', 'Chow Time',
       'Cauldron Filler (Assessment)', 'Balancing Act',
       'Crystal Caves - Level 2', 'Crystal Caves - Level 3',
       'Chicken Balancer (Activity)', 'Lifting Heavy Things',
       'Pan Balance', 'Happy Camel', 'Honey Cake',
       'Cart Balancer (Assessment)', 'Heavy, 

In [58]:
# none is landing page
# 3 worlds in total
train.world.unique()

array(['NONE', 'MAGMAPEAK', 'TREETOPCITY', 'CRYSTALCAVES'], dtype=object)

In [60]:
train.groupby("world")["type"].value_counts()

world         type      
CRYSTALCAVES  Game          2129478
              Activity       691564
              Assessment     353661
              Clip            57843
MAGMAPEAK     Activity      2407670
              Game          2396380
              Assessment     181925
              Clip            37712
NONE          Clip            23578
TREETOPCITY   Activity      1337494
              Game          1269435
              Assessment     389759
              Clip            64543
Name: type, dtype: int64

# Map of the whole app

In [68]:
train.groupby(["world", "type"])["title"].nunique().to_frame()

title
world        type             
CRYSTALCAVES Activity        2
             Assessment      2
             Clip            7
             Game            4
MAGMAPEAK    Activity        3
             Assessment      1
             Clip            3
             Game            4
NONE         Clip            1
TREETOPCITY  Activity        3
             Assessment      2
             Clip            9
             Game            3

In [72]:
print(f"""
event codes: 

{train.event_code.unique()}

and there are {train.event_code.nunique()} many events types

while titles for different 'missions' for all combinations of world * activity types
is: {train.groupby(["world", "type"])["title"].nunique().sum()}
      
Maybe the event codes are coding for different missions choice?
""")


event codes: 

[2000 3010 3110 4070 4090 4030 4035 4021 4020 4010 2080 2083 2040 2020
 2030 3021 3121 2050 3020 3120 2060 2070 4031 4025 5000 5010 2081 2025
 4022 2035 4040 4100 2010 4110 4045 4095 4220 2075 4230 4235 4080 4050]

and there are 42 many events types

while titles for different 'missions' for all combinations of world * activity types
is: 44
      
Maybe the event codes are coding for different missions choice?



In [9]:
train.installation_id.nunique()

17000

In [18]:
train_label.groupby("installation_id")["accuracy_group"].value_counts()

installation_id  accuracy_group
0006a69f         3                 3
                 0                 1
                 2                 1
0006c192         0                 1
                 2                 1
                                  ..
ffc90c32         3                 5
                 1                 1
ffd2871d         3                 1
ffeb0b1b         1                 2
                 0                 1
Name: accuracy_group, Length: 7203, dtype: int64

In [19]:
train_label[train_label.installation_id == "0006c192"]

,game_session,installation_id,title,num_correct,num_incorrect,accuracy,accuracy_group
5,197a373a77101924,0006c192,Cauldron Filler (Assessment),1,0,1.0,3
6,957406a905d59afd,0006c192,Bird Measurer (Assessment),1,1,0.5,2
7,b2297d292892745a,0006c192,Mushroom Sorter (Assessment),0,4,0.0,0


In [20]:
train_label.groupby(["installation_id", "title"])["accuracy_group"].max()

installation_id  title                       
0006a69f         Bird Measurer (Assessment)      3
                 Mushroom Sorter (Assessment)    3
0006c192         Bird Measurer (Assessment)      2
                 Cauldron Filler (Assessment)    3
                 Mushroom Sorter (Assessment)    0
                                                ..
ffc90c32         Chest Sorter (Assessment)       3
                 Mushroom Sorter (Assessment)    3
ffd2871d         Cauldron Filler (Assessment)    3
ffeb0b1b         Cauldron Filler (Assessment)    1
                 Mushroom Sorter (Assessment)    0
Name: accuracy_group, Length: 9386, dtype: int64

In [7]:
# game_session, each session kid starts
train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 11341042 entries, 0 to 11341041
Data columns (total 11 columns):
event_id           object
game_session       object
timestamp          object
event_data         object
installation_id    object
event_count        int64
event_code         int64
game_time          int64
title              object
type               object
world              object
dtypes: int64(3), object(8)
memory usage: 951.8+ MB


In [6]:
train_label.head()

,game_session,installation_id,title,num_correct,num_incorrect,accuracy,accuracy_group
0,6bdf9623adc94d89,0006a69f,Mushroom Sorter (Assessment),1,0,1.0,3
1,77b8ee947eb84b4e,0006a69f,Bird Measurer (Assessment),0,11,0.0,0
2,901acc108f55a5a1,0006a69f,Mushroom Sorter (Assessment),1,0,1.0,3
3,9501794defd84e4d,0006a69f,Mushroom Sorter (Assessment),1,1,0.5,2
4,a9ef3ecb3d1acc6a,0006a69f,Bird Measurer (Assessment),1,0,1.0,3


In [8]:
train_label.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 17690 entries, 0 to 17689
Data columns (total 7 columns):
game_session       17690 non-null object
installation_id    17690 non-null object
title              17690 non-null object
num_correct        17690 non-null int64
num_incorrect      17690 non-null int64
accuracy           17690 non-null float64
accuracy_group     17690 non-null int64
dtypes: float64(1), int64(3), object(3)
memory usage: 967.5+ KB


In [9]:
train_label.title

0        Mushroom Sorter (Assessment)
1          Bird Measurer (Assessment)
2        Mushroom Sorter (Assessment)
3        Mushroom Sorter (Assessment)
4          Bird Measurer (Assessment)
                     ...             
17685      Bird Measurer (Assessment)
17686    Cauldron Filler (Assessment)
17687    Cauldron Filler (Assessment)
17688    Mushroom Sorter (Assessment)
17689    Cauldron Filler (Assessment)
Name: title, Length: 17690, dtype: object

In [10]:
spec = pd.read_csv("specs.csv")

In [11]:
spec.head()

,event_id,info,args
0,2b9272f4,The end of system-initiated feedback (Correct)...,"[{""name"":""game_time"",""type"":""int"",""info"":""mill..."
1,df4fe8b6,The end of system-initiated feedback (Incorrec...,"[{""name"":""game_time"",""type"":""int"",""info"":""mill..."
2,3babcb9b,The end of system-initiated instruction event ...,"[{""name"":""game_time"",""type"":""int"",""info"":""mill..."
3,7f0836bf,The end of system-initiated instruction event ...,"[{""name"":""game_time"",""type"":""int"",""info"":""mill..."
4,ab3136ba,The end of system-initiated instruction event ...,"[{""name"":""game_time"",""type"":""int"",""info"":""mill..."


In [12]:
spec.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 386 entries, 0 to 385
Data columns (total 3 columns):
event_id    386 non-null object
info        386 non-null object
args        386 non-null object
dtypes: object(3)
memory usage: 9.2+ KB


In [14]:
spec.args[0]

'[{"name":"game_time","type":"int","info":"millisecond count since start of game"},{"name":"round","type":"int","info":"number of the current round when the event takes place or 0 if no round"},{"name":"level","type":"int","info":"number of the current level when the event takes place or 0 if no level"},{"name":"description","type":"string","info":"the text or description of the instruction"},{"name":"identifier","type":"string","info":"a unique identifier for this piece of instruction"},{"name":"media_type","type":"string","info":"the type of media that has just played:\\n\'audio\' || \'animation\' || \'other\'"},{"name":"duration","type":"int","info":"the duration of the media playback in milliseconds"},{"name":"event_count","type":"int","info":"session event counter"},{"name":"event_code","type":"int","info":"event class identifier"}]'

In [16]:
train.groupby("event_code")["event_id"].value_counts()

event_code  event_id
2000        27253bdc    183676
            77261ab5     11473
            7d093bf9     10196
            d2278a3b      7372
            4901243f      6341
                         ...  
4220        1340b8d7     18777
4230        ad148f58     31535
4235        85de926c     31526
5000        a6d66e51     21998
5010        71e712d8     21541
Name: event_id, Length: 384, dtype: int64

In [21]:
train.world.value_counts()

MAGMAPEAK       5023687
CRYSTALCAVES    3232546
TREETOPCITY     3061231
NONE              23578
Name: world, dtype: int64

In [23]:
train.groupby(["world"])["type"].value_counts()

world         type      
CRYSTALCAVES  Game          2129478
              Activity       691564
              Assessment     353661
              Clip            57843
MAGMAPEAK     Activity      2407670
              Game          2396380
              Assessment     181925
              Clip            37712
NONE          Clip            23578
TREETOPCITY   Activity      1337494
              Game          1269435
              Assessment     389759
              Clip            64543
Name: type, dtype: int64

In [24]:
train.groupby(["world","type"])["title"].value_counts()

world         type        title                        
CRYSTALCAVES  Activity    Chicken Balancer (Activity)       443156
                          Egg Dropper (Activity)            248408
              Assessment  Chest Sorter (Assessment)         190318
                          Cart Balancer (Assessment)        163343
              Clip        Crystal Caves - Level 2            14342
                          Crystal Caves - Level 1            12505
                          Crystal Caves - Level 3            12482
                          Lifting Heavy Things                5759
                          Balancing Act                       5522
                          Honey Cake                          3882
                          Heavy, Heavier, Heaviest            3351
              Game        Chow Time                        1150974
                          Pan Balance                       384857
                          Happy Camel                       311543
      

In [26]:
train.groupby(["world", "type", "title"])["event_id"].value_counts()

world         type      title                        event_id
CRYSTALCAVES  Activity  Chicken Balancer (Activity)  56bcd38d    151200
                                                     499edb7c     80192
                                                     4bb2f698     73820
                                                     cdd22e43     56784
                                                     ea321fb1     31342
                                                                  ...  
TREETOPCITY   Game      Crystals Rule                48349b14      3145
                                                     cc5087a3      2887
                                                     a1192f43      1989
                                                     93edfe2e       251
                                                     17ca3959         1
Name: event_id, Length: 403, dtype: int64

In [28]:
train.groupby(["world", "type", "title"])["event_code"].value_counts()

world         type      title                        event_code
CRYSTALCAVES  Activity  Chicken Balancer (Activity)  4030          151200
                                                     4020           80192
                                                     4070           73820
                                                     4035           56784
                                                     3010           31342
                                                                    ...  
TREETOPCITY   Game      Crystals Rule                2000            3145
                                                     4010            2887
                                                     4050            1989
                                                     4090             251
                                                     2010               1
Name: event_code, Length: 403, dtype: int64

In [44]:
train[(train.world == "TREETOPCITY") & (train.type == "Game") & (train.title == "Crystals Rule") & (train.event_code == 4030)].event_id.value_counts()

Series([], Name: event_id, dtype: int64)

In [40]:
train[(train.world == "TREETOPCITY") & (train.type == "Game") & (train.title == "Crystals Rule") & (train.event_code == 2010)].event_id.value_counts()

17ca3959    1
Name: event_id, dtype: int64

In [42]:
train[(train.world == "TREETOPCITY") & (train.type == "Game") & (train.title == "Crystals Rule") & (train.event_code == 4090)].event_id.unique()

array(['93edfe2e'], dtype=object)

In [43]:
train.title.nunique()

44

In [45]:
train_label.head()

,game_session,installation_id,title,num_correct,num_incorrect,accuracy,accuracy_group
0,6bdf9623adc94d89,0006a69f,Mushroom Sorter (Assessment),1,0,1.0,3
1,77b8ee947eb84b4e,0006a69f,Bird Measurer (Assessment),0,11,0.0,0
2,901acc108f55a5a1,0006a69f,Mushroom Sorter (Assessment),1,0,1.0,3
3,9501794defd84e4d,0006a69f,Mushroom Sorter (Assessment),1,1,0.5,2
4,a9ef3ecb3d1acc6a,0006a69f,Bird Measurer (Assessment),1,0,1.0,3


In [46]:
train_label.accuracy_group.unique()

array([3, 0, 2, 1])

In [49]:
big = pd.merge(train, train_label, how = 'inner', on = "installation_id")

In [61]:
s = pd.merge(train, train_label, how = 'left', on = "installation_id")

In [62]:
s.shape

(95428453, 17)

In [57]:
train_label.shape

(17690, 7)

In [64]:
train.installation_id.nunique()

17000

In [63]:
train[train.event_code == 4100]

,event_id,game_session,timestamp,event_data,installation_id,event_count,event_code,game_time,title,type,world
2228,25fa8af4,901acc108f55a5a1,2019-08-06T05:22:32.357Z,"{""correct"":true,""stumps"":[1,2,4],""event_count""...",0006a69f,44,4100,31011,Mushroom Sorter (Assessment),Assessment,TREETOPCITY
2308,14de4c5d,80d34a30c2998653,2019-08-06T05:24:50.323Z,"{""distance"":10,""target_distances"":[5,6,7,8,9,1...",0006a69f,76,4100,114370,Air Show,Game,TREETOPCITY
2335,14de4c5d,80d34a30c2998653,2019-08-06T05:25:11.292Z,"{""distance"":9,""target_distances"":[5,6,7],""corr...",0006a69f,103,4100,135341,Air Show,Game,TREETOPCITY
2375,14de4c5d,80d34a30c2998653,2019-08-06T05:25:37.207Z,"{""distance"":3,""target_distances"":[5,6,7],""corr...",0006a69f,143,4100,161258,Air Show,Game,TREETOPCITY
2409,14de4c5d,80d34a30c2998653,2019-08-06T05:26:01.055Z,"{""distance"":8,""target_distances"":[5,6,7],""corr...",0006a69f,177,4100,185103,Air Show,Game,TREETOPCITY
...,...,...,...,...,...,...,...,...,...,...,...
11338726,14de4c5d,26b2ccaacf21b135,2019-08-23T01:58:10.691Z,"{""distance"":9,""target_distances"":[6,7,8],""corr...",ffecb712,129,4100,200887,Air Show,Game,TREETOPCITY
11338735,14de4c5d,26b2ccaacf21b135,2019-08-23T01:58:29.249Z,"{""distance"":9,""target_distances"":[6,7,8],""corr...",ffecb712,138,4100,219447,Air Show,Game,TREETOPCITY
11338744,14de4c5d,26b2ccaacf21b135,2019-08-23T01:58:50.809Z,"{""distance"":4,""target_distances"":[6,7,8],""corr...",ffecb712,147,4100,241015,Air Show,Game,TREETOPCITY
11338753,14de4c5d,26b2ccaacf21b135,2019-08-23T01:59:14.049Z,"{""distance"":5,""target_distances"":[6,7,8],""corr...",ffecb712,156,4100,264252,Air Show,Game,TREETOPCITY


In [65]:
df = pd.merge(train, train_label, how = 'left', on = "game_session")

In [82]:
df.shape

(11341042, 17)

In [83]:
df.drop(columns = ['installation_id_y', 'title_y'], inplace = True)

In [85]:
#df = df.drop(columns=['installation_id_y', 'title_y'])
df = df.rename(columns={'installation_id_x': 'installation_id', 'game_session': 'session', 'title_x': 'title'})

In [86]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 11341042 entries, 0 to 11341041
Data columns (total 15 columns):
event_id           object
session            object
timestamp          object
event_data         object
installation_id    object
event_count        int64
event_code         int64
game_time          int64
title              object
type               object
world              object
num_correct        float64
num_incorrect      float64
accuracy           float64
accuracy_group     float64
dtypes: float64(4), int64(3), object(8)
memory usage: 1.4+ GB


In [88]:
train.shape

(11341042, 11)

In [87]:
df.to_csv("explore.csv")

In [67]:
df.columns

Index(['event_id', 'game_session', 'timestamp', 'event_data',
       'installation_id_x', 'event_count', 'event_code', 'game_time',
       'title_x', 'type', 'world', 'installation_id_y', 'title_y',
       'num_correct', 'num_incorrect', 'accuracy', 'accuracy_group'],
      dtype='object')

In [68]:
big_df = df[~df.accuracy_group.isna()]

In [79]:
spec.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 386 entries, 0 to 385
Data columns (total 3 columns):
event_id    386 non-null object
info        386 non-null object
args        386 non-null object
dtypes: object(3)
memory usage: 9.2+ KB


In [74]:
big_df.drop(columns = ['installation_id_y','title_y'])

KeyError: "['installation_id_y' 'title_y'] not found in axis"

In [80]:
big_df.shape

(865447, 15)